# Construction of Catalytic Potential Models
All models are reconstructed based on the information from Yurkovich et al., “Network-Level Allosteric Effects Are Elucidated by Detailing How Ligand-Binding Events Modulate Utilization of Catalytic Potentials.”

__URL:__ https://doi.org/10.1371/journal.pcbi.1006356

## Setup
#### Import packages

In [ ]:
from os import path 

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

import roadrunner as rr

import mass
from mass import MassConfiguration, Simulation
from mass.io import json, sbml
from mass.visualization import plot_time_profile

from table_export import export_csv_files_for_models

print("MASSpy version: " + mass.__version__)

#### Set configurations

In [ ]:
MASSCONFIGURATION = MassConfiguration()

MASSCONFIGURATION.decimal_precision = 12

# Configure the roadrunner options to prevent excessive logging output and increase performance
rr.Config.setValue(
    rr.Config.ROADRUNNER_DISABLE_PYTHON_DYNAMIC_PROPERTIES, False)
rr.Config.setValue(rr.Config.MAX_OUTPUT_ROWS, 1e6)

## Construct Non-personalized models
### Load Base Models
Load the following models:

\\(\text{Glycolysis}\\ \text{Hemoglobin (Hb)}\\ \text{Hexokinase 1 (HEX1)}\\ \text{Phosphofructokinase (PFK)}\\ \text{Pyruvate Kinase (PYK)}\\)

In [ ]:
# Function to print S matrix dimensions
def print_S_dimensions(model):
    print("{0}: Dim(S) = {1}x{2}".format(model.id, *model.S.shape))

# Function to load a model 
def load_model(model_id):
    filename = model_id + ".json"
    filepath = path.abspath(path.join("data", "models", "JSON", filename))
    model = json.load_json_model(filepath)
    print_S_dimensions(model)
    return model

model_ids = ["Glycolysis", "Hemoglobin", "HEX1", "PFK", "PYK"]
glycolysis, hemoglobin, HEX1, PFK, PYK = list(map(load_model, model_ids))

for enzyme_module in [HEX1, PFK, PYK]:
    for g in enzyme_module.groups:
        g.id = "_".join((enzyme_module.id, g.id))

### Load Concentration and Parameter Data

In [ ]:
data_filepath = path.abspath(path.join("data", "Non-personalized-data.xlsx"))
data_dict = pd.read_excel(
    data_filepath, sheet_name=["Concentrations", "Fluxes", "Equilibrium Constants", "PERCs"],
    index_col=0)

# Function to load a DataFrame into the necessary format
def load_dataframe(data_key, drop_units=False):
    df = data_dict[data_key]
    df = df.replace("-", np.nan)
    if drop_units:
        df = df.drop("Units", axis=1)
    df.columns = [col.replace(" + ", "_") for col in df.columns]
    return df

conc_df = load_dataframe("Concentrations", drop_units=True)
flux_df = load_dataframe("Fluxes", drop_units=True)
Keqs_df = load_dataframe("Equilibrium Constants", drop_units=False)
perc_df = load_dataframe("PERCs", drop_units=False)

all_models = []

#### Functions to update models with data

In [ ]:
# Function to update model concentrations
def update_concentrations(model, verbose=False):
    concentrations = {}
    boundary_conditions = {}
    boundary_compartment = "_" + list(MASSCONFIGURATION.boundary_compartment).pop()

    model_concs = conc_df[model.id]
    model_concs = model_concs.dropna()
    for met, conc in model_concs.to_dict().items():
        if met.endswith(boundary_compartment):
            boundary_conditions[met] = conc
        else:
            concentrations[met] = conc

    model.update_initial_conditions(concentrations, verbose=verbose)
    model.update_parameters(boundary_conditions, verbose=verbose)
    print(model.id + " concentrations updated.")
    
# Function to update model parameter
def update_parameter_type(model, parameter_type, verbose=False):
    param_df, key = {
        "fluxes": (flux_df, "Flux ID"),
        "equilibrium constants": (Keqs_df, "Keq ID"),
        "PERCs": (perc_df, "PERC ID")}[parameter_type]
    model_df = param_df[[key, model.id]]
    model_df = model_df.dropna()
    value_dict = {param_id: float(param_value) for rxn_id, (param_id, param_value) in model_df.iterrows()}
    model.update_parameters(value_dict, verbose=verbose)
    print(" ".join((model.id, parameter_type, "updated.")))

### Create Models and Export
#### Define functions

In [ ]:
# Function to add an EnzymeModule to a MassModel
def add_enzyme_to_model(model, enzyme_module):
    new_model = model.merge(enzyme_module, inplace=False)
    new_model.remove_reactions([new_model.reactions.get_by_id(enzyme_module.id)])
    return new_model

# Function to update model values and validate steady state
def update_model_and_simulate(model):
    print_S_dimensions(model)

    update_concentrations(model)
    update_parameter_type(model, "fluxes")
    update_parameter_type(model, "equilibrium constants")
    update_parameter_type(model, "PERCs")

    sim = Simulation(model, verbose=True)
    sim.find_steady_state(model, strategy="simulate", update_values=True,
                          verbose=False, decimal_precision=False,
                          steps=1e5, tfinal=1e4)

    conc_sol, flux_sol = sim.simulate(model, time=(0, 1e3, 1e4+1),
                                      verbose=False, decimal_precision=False)

    fig, ax = plt.subplots(nrows=1, ncols=1,
                           figsize=(6, 4),
                           constrained_layout=True);
    plot_time_profile(
        conc_sol, ax=ax, plot_function="loglog",
        xlabel="Time [hr]", ylabel="Conc. [mM]",
        title=model.id + " Concentration Profile");

# Function to save a model 
def save_model(model):
    # Export model as JSON
    json.save_json_model(
        mass_model=model, filename=path.abspath(
            path.join("data", "models", "JSON", model.id  + ".json")))
    # Export model as SBML
    sbml.write_sbml_model(mass_model=model, filename=path.abspath(
            path.join("data", "models", "SBML", model.id  + ".xml.zip")))
    print(" ".join(("Exported", model.id, "as JSON and SBML files.")))

#### No Regulation
##### Glycolysis

In [ ]:
model = glycolysis
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb

In [ ]:
glycolysis_hb = glycolysis.merge(hemoglobin, inplace=False)
glycolysis_hb.id = "Glycolysis_Hb"
model = glycolysis_hb
update_model_and_simulate(model)
save_model(model)
all_models += [model]

#### Regulation With One Enzyme
##### Glycolysis + PFK

In [ ]:
model = add_enzyme_to_model(glycolysis, PFK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + PYK

In [ ]:
model = add_enzyme_to_model(glycolysis, PYK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb + HEX1 

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, HEX1)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb + PFK 

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, PFK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb + PYK 

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, PYK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

#### Regulation With Two Enzymes
##### Glycolysis + PFK + PYK

In [ ]:
model = add_enzyme_to_model(glycolysis, PFK)
model = add_enzyme_to_model(model, PYK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb + PFK + PYK

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, PFK)
model = add_enzyme_to_model(model, PYK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb + HEX1 + PFK

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, HEX1)
model = add_enzyme_to_model(model, PFK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

##### Glycolysis + Hb + HEX1 + PYK

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, HEX1)
model = add_enzyme_to_model(model, PYK)
update_model_and_simulate(model)
save_model(model)
all_models += [model]

#### Regulation With Three Enzymes
##### Glycolysis + Hb + HEX1 + PFK + PYK

In [ ]:
model = add_enzyme_to_model(glycolysis_hb, HEX1)
model = add_enzyme_to_model(model, PFK)
model = add_enzyme_to_model(model, PYK)
update_model_and_simulate(model)
model.id = "Glycolysis_FKRM"
save_model(model)
all_models += [model]

In [ ]:
# Export value tables
export_csv_files_for_models(all_models, "for_non-personalized_model_construction_notebook")